In [ ]:
import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

from   datetime          import timedelta
from   datetime          import datetime


import pandas            as     pd

dt = datetime.now()

from   io                import StringIO

from arable_data_dict import Hourly_Data_Columns, Daily_Data_Columns, Soil_Data_Columns

Soil_Data_Columns = {
    "device"          :"Device ID",
    "time"            :"Time stamp",
    "sdi12_value_0"   :"5TE raw VWC (dielectric permittivity)",
    "sdi12_value_1"   :"5TE Bulk EC (dS/m)",
    "sdi12_value_2"   :"5TE Temp (C)",
    "analog"          :"EC-5 Sensor VWC (%)"
}

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

def arable_query(s, b, c, d, e, f, g):
    df = a.query( select = str(s),
                  format = str(b),
                 devices = c,
                 measure = str(d),
                   order = str(e),
                     end = f,
                   start = g,
                   limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

start_current_month = ym_to_dt(dt_to_ym(dt)).replace(day=1)
print(start_current_month)
print(dt_to_ym(start_current_month))

today = dt

import os
dirpath = 'C:/Users/ccber-arables/Box/'

def write_csv_file(sta, end, data_type, data_type_fn, folder, cm, m, remove, dev, loc, site): 
    logfile = 'Log_' + str(dt_to_ymd(today)) + '.txt'
    devicee = "['" + str(dev) + "']"
    fn = str(folder) + str(loc) + '_' + str(cm) + '_' + str(data_type_fn) + '.csv'
    print(fn)
    try:
        df = a.query(select = 'all', 
                     format = 'csv', 
                     devices = devicee, 
                     measure = str(data_type), 
                     order = "time", 
                     end = end, 
                     start = sta, 
                     limit = 100000000)
        
        df = StringIO(df)
        df = pd.read_csv(df, sep=',', error_bad_lines=False)
        while data_type_fn == "Soil":
            raw_VWC = df['sdi12_value_0'].to_list()
            cal_VWC = []
            for i in range(len(raw_VWC)):
                x = float(i)
                cal_VWC.append(float(4.3*10**(-6)*x**(3) - 5.5*10**(-4)*x**(2) + 2.92*10**(-2)*x - 5.3*10**(-2)))
            df = df.dropna(axis='columns', thresh=2)#how='all')
            df.insert(1, 'Habitat', loc)
            df.insert(1, 'Site', site)
            tm = df['time'].to_list()
            date_stamp = []
            time_stamp = []
            for i in range(len(tm)):
                date_stamp.append(str(tm[i])[:10])
                time_stamp.append(str(tm[i])[11:][:8])
            df.insert(1, 'Time', time_stamp)
            df.insert(1, 'Date', date_stamp) 
            for i in remove:
                print("Remove " + i)
                try:                
                    df = df.drop(i, axis='columns')
                except:
                    pass
            df = df.rename(mapper=m, axis='columns')
            df.insert(6, '5TE Calibrated VWC (m^3/m^3)', cal_VWC)
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
            open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
        else:
            df = df.dropna(axis='columns', thresh=2)#how='all')
            df.insert(1, 'Habitat', loc)
            df.insert(1, 'Site', site)
            tm = df['time'].to_list()
            date_stamp = []
            time_stamp = []
            for i in range(len(tm)):
                date_stamp.append(str(tm[i])[:10])
                time_stamp.append(str(tm[i])[11:][:8])
            df.insert(1, 'Time', time_stamp)
            df.insert(1, 'Date', date_stamp) 
            for i in remove:
                print("Remove " + i)
                df = df.drop(i, axis='columns')
            df = df.rename(mapper=m, axis='columns')
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
            open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
    except:
        open(logfile, 'a+').write('Failure writing ' + fn + '\n')
        #pass     
device_ids = pd.read_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")['ID'].values.tolist()    
locations  = pd.read_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")['Location'].values.tolist()  


#----Site
#--------Habitat
#---------------Year
#-------------------Daily
#-------------------Hourly
#-------------------Soil
   
Sensor_ID            = ['A000229','A000318','A000369','A000465','A000466','A000502','A000507','A000513','A000522','A000536','A000546','A000563']

Site                 = ['Campus_Lagoon','Campus_Lagoon','North_Campus_Open_Space','Campus_Lagoon','North_Campus_Open_Space','North_Campus_Open_Space','Nipomo_Lupine_Project','Nipomo_Lupine_Project','Nipomo_Lupine_Project','Campus_Lagoon','North_Campus_Open_Space','North_Campus_Open_Space']
Habitat              = ['Annual_Grass','Coastal_Sage_Scrub','Ventura_Marsh_Milk-Vetch','Campus_Point_Iceplant','Salt_Marsh','North_Slope_Oaks','Black_Lake_Gentle_South','Black_Lake_North_Facing','Philips_66','2016_Burn_Plot','2017_Stipa_Drillseed','Stipa_Loam']

Current_Year         = start_current_month.strftime("%Y")

Columns_to_Remove    = [['location','time','lat','long'],['location','time','lat','long'],['location','time','sdi12_version','sdi12_vendor_id','sdi12_sensor_model','sdi12_sensor_version','sdi12_sensor_sn','pulse_interval','pulse_state','pulse_falling','pulse_rising']]
Data_Type            = ["Daily", "Hourly", "Soil"]
Data_Types           = ["daily", "hourly", "aux_raw"]

Column_Names         = [Daily_Data_Columns, Hourly_Data_Columns, Soil_Data_Columns]


for i in range(len(Sensor_ID)):
    Sensor = Sensor_ID[i]
    Habi   = Habitat[i]
    print(Habi)
    Folder_Name_A = str(dirpath + Site[i] + '/' + Habitat[i] + '/' + str(Current_Year) + '/')
    for j in range(len(Data_Type)):
        Folder_Name = str(Folder_Name_A + Data_Type[j] + '/')
        if not os.path.exists(Folder_Name):
            os.makedirs(Folder_Name)
            print "Directory " , Folder_Name ,  " Created "
        else:    
            print "Directory " , Folder_Name ,  " already exists"  
            pass
        write_csv_file(start_current_month, today, Data_Types[j], Data_Type[j], Folder_Name, dt_to_ym(start_current_month), Column_Names[j], Columns_to_Remove[j], Sensor, Habi, Site[i])
